In [1]:
#dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import csv
import os
import pymongo

#set up mongodb server
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.movies
movies = db.movies
#create engine, establish sqlite file to push data into
engine = create_engine('sqlite:///movie.sqlite', echo=False)

#establish paths for both csvs to be used
rt_path = 'rotten_tomatoes_movies.csv'
imdb_path= 'imdb.csv'

#imdb csv has to be bypassed ,as original data had commas in titles, thus making the information mvoe around
#err_bad_lines=false skips those lines
imdb_rating = pd.read_csv(imdb_path, encoding ='UTF-8', error_bad_lines=False)
rt_rating = pd.read_csv(rt_path)

b'Skipping line 66: expected 44 fields, saw 46\nSkipping line 111: expected 44 fields, saw 45\nSkipping line 198: expected 44 fields, saw 45\nSkipping line 222: expected 44 fields, saw 46\nSkipping line 278: expected 44 fields, saw 45\nSkipping line 396: expected 44 fields, saw 45\nSkipping line 403: expected 44 fields, saw 45\nSkipping line 421: expected 44 fields, saw 45\nSkipping line 437: expected 44 fields, saw 45\nSkipping line 462: expected 44 fields, saw 46\nSkipping line 491: expected 44 fields, saw 45\nSkipping line 515: expected 44 fields, saw 45\nSkipping line 529: expected 44 fields, saw 45\nSkipping line 530: expected 44 fields, saw 45\nSkipping line 558: expected 44 fields, saw 45\nSkipping line 623: expected 44 fields, saw 45\nSkipping line 646: expected 44 fields, saw 45\nSkipping line 663: expected 44 fields, saw 46\nSkipping line 713: expected 44 fields, saw 45\nSkipping line 730: expected 44 fields, saw 47\nSkipping line 791: expected 44 fields, saw 45\nSkipping lin

In [2]:
#creates column that pulls in date based on whether or not it exists
#in this case, if it was never released in theaters, it uses the streaming date
rt_rating['year']= np.where(rt_rating['in_theaters_date'].isnull(), rt_rating['on_streaming_date'], rt_rating['in_theaters_date'])

#converts the year to actual datetime to allow for the year to be pulled
rt_rating['year'] = pd.to_datetime(rt_rating['year'])

#pulls year only for joining purposes
rt_rating['year'] = rt_rating['year'].dt.year

#removes any rotten tomato movies that does not have a rating
rt_rating = rt_rating.dropna()

#movie to int format to remove decimals
rt_rating['year'] = rt_rating['year'].astype(int)

In [3]:
rt_rating

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count,year
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,Rotten,49,144,Spilled,53.0,254287.0,38,71,73,2010
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Certified Fresh,86,140,Upright,64.0,11567.0,43,121,19,2010
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Certified Fresh,100,51,Upright,97.0,105000.0,6,51,0,1957
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Fresh,89,27,Upright,74.0,68860.0,5,24,3,1954
5,/m/10000_bc,"10,000 B.C.",A young outcast from a primitive tribe is forc...,With attention strictly paid to style instead ...,https://resizing.flixster.com/x41Imwhe6xccM0Zl...,PG-13,"Action & Adventure, Classics, Drama",Roland Emmerich,"Roland Emmerich, Harald Kloser","Steven Strait, Camilla Belle, Cliff Curtis, Jo...",...,Rotten,8,148,Spilled,37.0,411087.0,36,12,136,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16629,/m/zookeeper,Zookeeper,"In Zookeeper, the animals at the Franklin Park...",Zookeeper smothers Kevin James's with a sodden...,https://resizing.flixster.com/KJ7WDE1seqGxY4op...,PG,"Comedy, Romance","Frank Coraci, Walt Becker","Jay Scherick, David Ronn, Nick Bakay, Rock Reu...","Kevin James, Rosario Dawson, Ken Jeong, Leslie...",...,Rotten,14,131,Spilled,41.0,44664.0,34,18,113,2011
16630,/m/zoolander,Zoolander,"Comic actor Ben Stiller co-wrote, directed, an...","A wacky satire on the fashion industry, Zoolan...",https://resizing.flixster.com/KY3_dCjyegEBq8nt...,PG-13,"Comedy, Special Interest",Ben Stiller,"Drake Sather, John Hamburg, Ben Stiller","Ben Stiller, Owen Wilson, Will Ferrell, Christ...",...,Fresh,64,137,Upright,80.0,944701.0,33,87,50,2001
16631,/m/zoolander_2,Zoolander 2,Ben Stiller returns both in front of and behin...,Zoolander No. 2 has more celebrity cameos than...,https://resizing.flixster.com/zJDLjddGcQaKAtdn...,PG-13,Comedy,Ben Stiller,"Justin Theroux, Ben Stiller, Nicholas Stoller,...","Ben Stiller, Owen Wilson, Will Ferrell, Penelo...",...,Rotten,23,226,Spilled,20.0,41699.0,47,51,175,2016
16632,/m/zoom_2006,Zoom,Former superhero Jack is called back to work t...,Lacking the punch and good cheer of The Incred...,https://resizing.flixster.com/DtmbcT7kLiZBOXFu...,PG,"Action & Adventure, Comedy, Kids & Family",Peter Hewitt,"Adam Rifkin, David Berenbaum","Tim Allen, Courteney Cox, Chevy Chase, Spencer...",...,Rotten,3,66,Spilled,33.0,11327.0,19,2,64,2006


In [4]:
#pulls in only the columns wanted from rt data
rt_rating = rt_rating[["movie_title","year",'genre','directors','tomatometer_rating','tomatometer_count','audience_rating','audience_count']]

#pulls away year data showing in the IMDB movie title
imdb_rating['title']=imdb_rating['title'].str.replace(r"\(.*\)","")

#pulls in only the columns wanted from the database
imdb_rating = imdb_rating[["title",'wordsInTitle', 'imdbRating', 'ratingCount','duration', 'year']]

In [5]:
#renames columns for common reference
rt_rating = rt_rating.rename(columns={"tomatometer_rating": "rt_critic_rating", "tomatometer_count": "rt_critic_count", "audience_rating": "rt_audience_rating", "audience_count": "rt_audience_count"}, errors="raise")

imdb_rating = imdb_rating.rename(columns = {"title":"movie_title"})


In [6]:
#converts movie title to lower in order to avoid uppercase issues
imdb_rating['lower']=imdb_rating["movie_title"].str.lower()
rt_rating['lower']=rt_rating["movie_title"].str.lower()

#remove spaces to ensure effective joins
rt_rating['lower']=rt_rating['lower'].replace(' ','',regex=True)
imdb_rating['lower']=imdb_rating['lower'].replace(' ','',regex=True)

In [7]:
#merges both dfs together
movies_df =rt_rating.merge(imdb_rating,on=['lower','year'])

In [8]:
#converts imdb rating to be in similar format to rotten tomatoes ratings
movies_df['adjusted_imdb_rating'] =movies_df['imdbRating']*10  

In [9]:
#get average rating based on three available sets of data
movies_df['average_rating'] = movies_df[['rt_critic_rating','rt_audience_rating','adjusted_imdb_rating']].mean(axis=1)

In [10]:
#removes unnecessary columns
movies_df = movies_df.drop(columns =['movie_title_y','lower','duration'])

In [11]:
#renames column for easy reference
movies_df = movies_df.rename(columns = {"movie_title_x":"title"})
#rounds average rating to 2 decimals
movies_df['average_rating']= movies_df['average_rating'].round(2)

In [12]:
#writes to csv
movies_df.to_csv('movies.csv')

In [13]:
#writes to sqlite data
movies_df.to_sql('movies', con=engine, if_exists = 'replace')

In [27]:
#set up to read off of newly created CSV to put into MongoDB as well
file = open(os.path.join('movies.csv'))
reader = csv.reader(file, delimiter=',')
data = [row for row in reader]
for row in data:
    try:
        movie_title = row[1]
        year = row[2]
        genre = row[3]
        directors = row[4]
        imdb_rating = row[10]
        imdb_rating_adjusted = row[12]
        imdb_rating_count = row[11]
        rt_critic_rating = row[5]
        rt_critic_count = row[6]
        rt_audience_rating = row[7]
        rt_audience_count = row[8]
        average_rating = row[13]
        movies.insert_one(
            {
                'movie_title': movie_title,
                'year': year,
                'genre': genre,
                'directors': directors,
                'imdb_rating': rt_critic_rating,
                'imdb_rating_adjusted': imdb_rating_adjusted,
                'imdb_rating_count': imdb_rating_count,
                'rt_critic_rating': rt_critic_rating,
                'rt_critic_count': rt_critic_count,
                'rt_audience_rating': rt_audience_rating,
                'rt_audience_count': rt_audience_count,
                'average_rating': average_rating
             }
         )
    except (KeyError, IndexError):
        print(f"{movie_title} unable to be written.")

title
Please Give
The Lost City
Running Scared
Ultraviolet
Oliver Twist
Stay Alive
Once in a Lifetime: The Extraordinary Story of the New York Cosmos
Slipstream
The Happening
The Midnight Meat Train
The Lucky Ones
Run Fatboy Run
Whiteout
Saawariya
Appaloosa
[Rec]
The Box
21
The Road
Public Enemies
Sin Nombre
Push
Felon
Chocolate
Splinter
Shrink
Repo Men
Winter's Bone
The Kids Are All Right
Predators
Bye Bye Birdie
Casablanca
Casino Royale
Cleopatra
Dracula
Dressed to Kill
Fahrenheit 451
Footloose
Frantic
Frosty the Snowman
Glory
Hairspray
Hamlet
Intolerance
Laura
Lolita
M
Mannequin
Marty
Meteor
Metropolis
Moby Dick
Point Blank
Poltergeist
Red Heat
Red Sonja
Road House
Robin Hood
RoboCop
Rocky
Sabrina
Shaft
Spartacus
A Star Is Born
A Star is Born
Tommy
Mystery Train
Malcolm X
Stalker
Gettysburg
Pocahontas
Braveheart
Kicking and Screaming
To Die For
Casino
Jumanji
Richard III
Heat
Sabrina
Heavy Metal
Sudden Death
Othello
Twister
Independence Day
Matilda
Jack
Freeway
Michael
Hamlet
Get Ca

Material Girls
Maurice
Max
Max Payne
McCabe & Mrs. Miller
The Mechanic
Mega Shark vs. Giant Octopus
Megamind
Melancholia
Memento
Men in Black
Men of Honor
Michael Clayton
Michael Collins
Mickey Blue Eyes
Midnight in Paris
A Mighty Wind
Milk
Miller's Crossing
Million Dollar Baby
The Million Dollar Hotel
Minority Report
The Miracle of Morgan's Creek
MirrorMask
Misery
Miss Julie
Mission: Impossible
Mission: Impossible III
Mission to Mars
Money Train
Monkeybone
Monster
Monster House
Monsters
Moon
Moonrise Kingdom
Morning Glory
Mortal Kombat
Moulin Rouge!
Mr. & Mrs. Smith
Mr. Deeds
Mr. Woodcock
Mrs. Miniver
Mulan
Mulholland Drive
My Blueberry Nights
My Name Is Khan
My Summer of Love
My Week with Marilyn
Mystery Science Theater 3000: The Movie
Mystic River
Nacho Libre
Napoleon Dynamite
Narc
Nashville
National Security
Natural Born Killers
Nebraska
Need For Speed
Network
Next
A Night at the Roxbury
Nine
Nixon
No Country for Old Men
No Man's Land
Non-Stop
Norbit
North
Northfork
Notting Hill
No